This notebook describes steps for installing the necessary software and genotyping TRs using using HipSTR, GangSTR, ExpansionHunter, and adVNTR. EnsembleTR is used to generate consensus TR genotypes from genotypes obtained from TR genotypers

# Obtain data

In [ ]:
# download toy dataset, about 2.09 GB
wget https://figshare.com/ndownloader/files/45851871

In [ ]:
# extract files from the archive 
tar -xvf example_data.tar.gz

In [ ]:
# create output directory
mkdir -p output/{hipstr_output,advntr_output/advntr_dir,gangstr_output,eh_output,ensembletr_output}

tree output

# Installing tools

## bcftools and tabix

In [ ]:
# steps to install bcftools are described here, https://samtools.github.io/bcftools/howtos/install.html

git clone --recurse-submodules https://github.com/samtools/htslib.git
git clone https://github.com/samtools/bcftools.git
cd bcftools
make

## HipSTR

In [ ]:
# HipSTR

# install these dependencies if you do not have them by uncommenting the line below

# apt install g++ make zlib1g-dev libhts-dev libbz2-dev liblzma-dev

git clone https://github.com/gymrek-lab/HipSTR.git
cd HipSTR
make version && make

In [ ]:
./HipSTR --help
cd ..

## ExpansionHunter

In [ ]:
# ExpansionHunter
version=v5.0.0-linux_x86_64

wget https://github.com/Illumina/ExpansionHunter/releases/download/v5.0.0/ExpansionHunter-v5.0.0-linux_x86_64.tar.gz

tar -xzvf ExpansionHunter-v5.0.0-linux_x86_64.tar.gz


In [ ]:
# The ExpansionHunter executable can be found in 
ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter --help

## GangSTR

In [ ]:
 # GangSTR

mamba create -y --name gangstr
mamba activate gangstr
mamba config --add channels conda-forge
mamba config --add channels bioconda
mamba install -y -c conda-forge -c bioconda gangstr

In [ ]:
# this step maybe optional
# only run if you get this error
# GangSTR: error while loading shared libraries: libgsl.so.25: cannot open shared object file: No such file or directory
mamba install -y conda-forge::gsl

In [ ]:
GangSTR --help

mamba deactivate

## adVNTR

In [ ]:
# advntr
mamba create -y --name advntr
mamba activate advntr
mamba config --add channels conda-forge
mamba config --add channels bioconda
mamba install -y -c conda-forge -c bioconda advntr

In [ ]:
# this step is optional
# run if the step below returns
# WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'

mamba install -y mkl-service

In [ ]:
advntr 
mamba deactivate

## EnsembleTR

In [ ]:
# ensembleTR
git clone https://github.com/gymrek-lab/EnsembleTR.git
cd EnsembleTR
python3 setup.py install --user

In [ ]:
EnsembleTR --version
cd ..

## TRtools

In [ ]:
git clone https://github.com/gymrek-lab/TRTools.git
cd TRTools/
python3 -m pip install --upgrade pip
python3 -m pip install -e .
cd ..

## vcftools

In [ ]:
git clone https://github.com/vcftools/vcftools.git
cd vcftools
./autogen.sh
./configure
make
sudo make install

cd ..

In [ ]:
# check software
./check_software.sh statSTR associaTR compareSTR dumpSTR mergeSTR prancSTR qcSTR simTR vcftools | grep -i 'not installed'

# Genotyping STRs and preprocessing VCF files

## HipSTR

In [ ]:
# HipSTR

populations=("africa" "europe" "east_asia" "south_asia" "america")
reference_genome="references/reference_chroms.fa"

for population in "${populations[@]}"; do
    
    bam_files=$(cat str_resources/"${population}".txt | tr "\n" ",")

    HipSTR/HipSTR --bams $bam_files \
        --fasta $reference_genome \
        --regions str_resources/hipstr_reference.bed \
        --str-vcf output/hipstr_output/"${population}"_hipstr.vcf.gz \
        --log output/hipstr_output/"${population}"_strs.log \
        --viz-out output/hipstr_output/"${population}"_strs.viz.gz \
        --output-gls --output-pls --def-stutter-model
        
    zcat output/hipstr_output/"${population}"_hipstr.vcf.gz | vcf-sort -c | bgzip -c > output/hipstr_output/"${population}"_hipstr_sorted.vcf.gz
    tabix -p vcf output/hipstr_output/"${population}"_hipstr_sorted.vcf.gz
    
done

### Understanding HipSTR output

In [ ]:
vcftools --gzvcf output/hipstr_output/africa_hipstr.vcf.gz --snp TIMM10 --indv NA20357 --recode --recode-INFO-all --stdout | tail -n 2 | datamash transpose

### Visualising HipSTR alignments

In [ ]:
# index the file with visualisation
tabix -p vcf output/hipstr_output/africa_strs.viz.gz

In [ ]:
# visualising alignments at chr17:51831668 for Africa super population
## africa
HipSTR/VizAlnPdf output/hipstr_output/africa_strs.viz.gz chr17 51831668 NA20357 viz_NA20357 3 


In [ ]:
convert -density 300 -quality 90 viz_NA20357.pdf -resize 25% NA20357.png
# display HG002330.png

![alignment](./NA20357.png)

## ExpansionHunter

In [ ]:
reference_genome="references/reference_chroms.fa"
output_dir=output/eh_output
male_list=str_resources/bams_male.txt
female_list=str_resources/bams_female.txt
eh_reference_strs=str_resources/eh_reference.json

# males
for sample_bam in $(cat $male_list); do
        
    sample_id=$(basename $sample_bam | cut -f1 -d".")        

    ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter \
            --threads 32 \
            --reads $sample_bam \
            --reference $reference_genome \
            --variant-catalog $eh_reference_strs \
            --output-prefix $output_dir/$sample_id \
            --sex male \
            --log-level trace

    # eh vcf files do not have contigs listed in the header
    cat output/eh_output/"${sample_id}".vcf | vcf-sort -c | bgzip -c > output/eh_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${sample_id}"_sorted.vcf.gz

done


# females
for sample_bam in $(cat $female_list); do
    
    sample_id=$(basename $sample_bam | cut -f1 -d".")        

    ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter \
            --threads 32 \
            --reads $sample_bam \
            --reference $reference_genome \
            --variant-catalog $eh_reference_strs \
            --output-prefix $output_dir/$sample_id \
            --sex female \
            --log-level trace
                
    # eh vcf files do not have contigs listed in the header
    cat output/eh_output/"${sample_id}".vcf | vcf-sort -c | bgzip -c > output/eh_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${sample_id}"_sorted.vcf.gz
    
done

In [ ]:
# merging sample VCFs into population VCFs

populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    # comma-seperated list of vcfs
    sample_vcfs=$(ls -1 output/eh_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_ids.csv | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'eh' \
        --out output/eh_output/"${population}"_eh
        
    cat output/eh_output/"${population}"_eh.vcf | vcf-sort -c | bgzip -c > output/eh_output/"${population}"_eh_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${population}"_eh_sorted.vcf.gz

done


## GangSTR

In [ ]:
-c mamba activate gangstr

reference_genome="references/reference_chroms.fa"
output_dir=output/gangstr_output
male_list=str_resources/bams_male.txt
female_list=str_resources/bams_female.txt
gangstr_reference_strs=str_resources/gangstr_reference.bed

# male samples
for sample_bam in $(cat $male_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    GangSTR --bam $sample_bam \
            --bam-samps $sample_id \
            --regions $gangstr_reference_strs \
            --samp-sex M \
            --ref $reference_genome \
            --out $output_dir/$sample_id \
            --insertmean 1 \
            --insertsdev 1 \
            --minmatch 1 \
            --min-sample-reads 1

    cat output/gangstr_output/"${sample_id}".vcf | vcf-sort -c | bgzip -c > output/gangstr_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/gangstr_output/"${sample_id}"_sorted.vcf.gz

done


#female samples
for sample_bam in $(cat $female_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    GangSTR --bam $sample_bam \
            --bam-samps $sample_id \
            --regions $gangstr_reference_strs \
            --samp-sex F \
            --ref $reference_genome \
            --out $output_dir/$sample_id \
            --insertmean 1 \
            --insertsdev 1 \
            --minmatch 1 \
            --min-sample-reads 1

    cat output/gangstr_output/"${sample_id}".vcf | vcf-sort -c | bgzip -c > output/gangstr_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/gangstr_output/"${sample_id}"_sorted.vcf.gz
            

done

mamba deactivate

In [ ]:
# merging sample VCFs to population VCF
populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    sample_vcfs=$(ls -1 output/gangstr_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_ids.csv | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'gangstr' \
        --out output/gangstr_output/"${population}"_gangstr
        
        
    cat output/gangstr_output/"${population}"_gangstr.vcf | vcf-sort -c | bgzip -c > output/gangstr_output/"${population}"_gangstr_sorted.vcf.gz
    tabix -p vcf output/gangstr_output/"${population}"_gangstr_sorted.vcf.gz
    
done

## adVNTR

In [ ]:
# view VNTR ID of CSTB gene

mamba activate advntr

advntr viewmodel --gene CSTB --pattern GCGCGGGGCGGG --models str_resources/vntr_data/hg19_selected_VNTRs_Illumina.db

In [ ]:
advntr_model=str_resources/vntr_data/hg19_selected_VNTRs_Illumina.db
output_dir=output/advntr_output
advntr_dir=output/advntr_output/advntr_dir
bam_list=str_resources/bam_list.txt


# main script
for sample_bam in $(cat $bam_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    advntr genotype \
            --vntr_id 301645 \
            --alignment_file $sample_bam \
            --outfmt vcf \
            --outfile $output_dir/"${sample_id}".vcf \
            --models $advntr_model \
            --working_directory $advntr_dir
    
    cat $output_dir/"${sample_id}".vcf | vcf-sort -c | bgzip -c > $output_dir/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/advntr_output/"${sample_id}"_sorted.vcf.gz


done

mamba deactivate

In [ ]:
populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    # comma-seperated list of vcfs
    sample_vcfs=$(ls -1 output/advntr_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_ids.csv | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'advntr' \
        --out output/advntr_output/"${population}"_advntr

    cat output/advntr_output/"${population}"_advntr.vcf | vcf-sort -c | bgzip -c > output/advntr_output/"${population}"_advntr_sorted.vcf.gz
    tabix -p vcf output/advntr_output/"${population}"_advntr_sorted.vcf.gz


done

## Merging population VCFs from each tools

In [ ]:
# gangstr
mergeSTR \
    --vcfs output/gangstr_output/africa_gangstr_sorted.vcf.gz,output/gangstr_output/east_asia_gangstr_sorted.vcf.gz,output/gangstr_output/south_asia_gangstr_sorted.vcf.gz,output/gangstr_output/america_gangstr_sorted.vcf.gz,output/gangstr_output/europe_gangstr_sorted.vcf.gz \
    --vcftype 'gangstr' \
    --out output/ensembletr_output/gangstr_strs

cat output/ensembletr_output/gangstr_strs.vcf | vcf-sort -c | bgzip -c > output/ensembletr_output/gangstr_strs_sorted.vcf.gz
tabix -p vcf output/ensembletr_output/gangstr_strs_sorted.vcf.gz

# expansionhunter
mergeSTR \
    --vcfs output/eh_output/africa_eh_sorted.vcf.gz,output/eh_output/east_asia_eh_sorted.vcf.gz,output/eh_output/south_asia_eh_sorted.vcf.gz,output/eh_output/america_eh_sorted.vcf.gz,output/eh_output/europe_eh_sorted.vcf.gz \
    --vcftype 'eh' \
    --out output/ensembletr_output/eh_strs

cat output/ensembletr_output/eh_strs.vcf | vcf-sort -c | bgzip -c > output/ensembletr_output/eh_strs_sorted.vcf.gz
tabix -p vcf output/ensembletr_output/eh_strs_sorted.vcf.gz

#advntr
mergeSTR \
    --vcfs output/advntr_output/africa_advntr_sorted.vcf.gz,output/advntr_output/east_asia_advntr_sorted.vcf.gz,output/advntr_output/south_asia_advntr_sorted.vcf.gz,output/advntr_output/america_advntr_sorted.vcf.gz,output/advntr_output/europe_advntr_sorted.vcf.gz \
    --vcftype 'advntr' \
    --out output/ensembletr_output/advntr_strs 
        
cat output/ensembletr_output/advntr_strs.vcf | vcf-sort -c | bgzip -c > output/ensembletr_output/advntr_strs_sorted.vcf.gz
tabix -p vcf output/ensembletr_output/advntr_strs_sorted.vcf.gz

# hipstr
mergeSTR \
    --vcfs output/hipstr_output/africa_hipstr_sorted.vcf.gz,output/hipstr_output/east_asia_hipstr_sorted.vcf.gz,output/hipstr_output/south_asia_hipstr_sorted.vcf.gz,output/hipstr_output/america_hipstr_sorted.vcf.gz,output/hipstr_output/europe_hipstr_sorted.vcf.gz \
    --vcftype 'hipstr' \
    --out output/ensembletr_output/hipstr_strs

cat output/ensembletr_output/hipstr_strs.vcf | vcf-sort -c | bgzip -c > output/ensembletr_output/hipstr_strs_sorted.vcf.gz
tabix -p vcf output/ensembletr_output/hipstr_strs_sorted.vcf.gz

## EnsembleTR consensus genotyping

In [ ]:
reference_genome="references/reference_chroms.fa"

EnsembleTR \
    --ref $reference_genome \
    --vcfs output/ensembletr_output/advntr_strs_sorted.vcf.gz,output/ensembletr_output/eh_strs_sorted.vcf.gz,output/ensembletr_output/gangstr_strs_sorted.vcf.gz,output/ensembletr_output/hipstr_strs_sorted.vcf.gz \
    --out output/ensembletr_output/ensembletr.vcf
    
cat output/ensembletr_output/ensembletr.vcf | vcf-sort -c | bgzip -c > output/ensembletr_output/ensembletr_sorted.vcf.gz
tabix -p vcf output/ensembletr_output/ensembletr_sorted.vcf.gz